In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install -U cloudpickle

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [3]:
df = 'ATTRITION_TABLE'


In [4]:
sf_df = my_session.sql("select * from {}".format(df))

In [5]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [6]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                                        0
TENURE_MONTHS                                      0
BIRTH_YEAR                                         0
AGE                                                0
SENIORITY                                          0
SCHOOL_ENDDATE                                     0
MONTHS_AFTER_COLLEGE                               0
PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH        0
PEOPLE_JOINED_AND_NEVER_LEFT                       0
POPULATION                                         0
CHURN_OTHER                                        0
CHURN_F                                            0
SUM_OF_TENURE                                      0
SUM_OF_AGE                                         0
JOB_STARTDATE                                      0
JOB_ENDDATE                                    90076
SALARY                                             0
JOB_SATISFACTION                                   0
PERCENTAGE_SALARY_HIKE                        

In [7]:
Original_df = pandas_df.dropna()

In [8]:
print(Original_df.isnull().sum())

EMPLOYEE_ID                                    0
TENURE_MONTHS                                  0
BIRTH_YEAR                                     0
AGE                                            0
SENIORITY                                      0
SCHOOL_ENDDATE                                 0
MONTHS_AFTER_COLLEGE                           0
PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH    0
PEOPLE_JOINED_AND_NEVER_LEFT                   0
POPULATION                                     0
CHURN_OTHER                                    0
CHURN_F                                        0
SUM_OF_TENURE                                  0
SUM_OF_AGE                                     0
JOB_STARTDATE                                  0
JOB_ENDDATE                                    0
SALARY                                         0
JOB_SATISFACTION                               0
PERCENTAGE_SALARY_HIKE                         0
PERFORMANCE_RATING                             0
OVER_TIME           

In [9]:
df = Original_df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_VALUE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","CHURN_F","SUM_OF_TENURE","SUM_OF_AGE"], axis = 1)

In [10]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE","SEX", "ETHNICITY","ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE","COUNTRY",
                       "NEW_DEVICE_INDICATOR","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN"]
DROPPED_COLUMNS = ["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_VALUE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","CHURN_F","SUM_OF_TENURE","SUM_OF_AGE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [11]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in Original_df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in Original_df.columns]
 
# Split data into features and labels
X = Original_df[feature_columns + DROPPED_COLUMNS]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
X_test

,MAPPED_ROLE,ETHNICITY,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,COUNTRY,BUSINESS_TRAVEL,...,JOB_ENDDATE,SCHOOL_ENDDATE,CHURN_VALUE,PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH,PEOPLE_JOINED_AND_NEVER_LEFT,POPULATION,CHURN_OTHER,CHURN_F,SUM_OF_TENURE,SUM_OF_AGE
265369,Software Development Engineer,Hispanic,Investment,Public,BridgePoint Capital,Orlando,Florida,5-10 miles,United States,Travel Rarely,...,2019-10-01,2014-01-01,FALSE,0,0,0,0,0,0,0
141833,Technical Support Associate,API,Technology,Public,NextGen Innovations,Rochester,New York,>10 miles,United States,Travel Rarely,...,2021-04-01,2010-01-01,TRUE,0,0,0,0,0,0,0
234366,Sales Development Representative,Hispanic,Technology,Public,SecureTech Solutions,Orlando,Florida,<2 miles,United States,Travel Rarely,...,2023-03-01,2019-01-31,FALSE,0,0,0,0,0,0,0
95853,Financial Analyst,API,Telecommunications,Public,Streamline Networks,Cleveland,Ohio,>10 miles,United States,Travel Frequently,...,2010-11-01,2007-01-01,TRUE,0,0,0,0,0,0,0
88598,Solutions Architect,Hispanic,Technology,Public,Pinnacle Technologies,Cleveland,Ohio,<2 miles,United States,Travel Frequently,...,2011-08-01,1967-01-31,FALSE,0,31,31,0,17,3224,1023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227011,Corporate Counsel,White,Telecommunications,Public,Streamline Networks,Madison,Wisconsin,<2 miles,United States,Travel Rarely,...,2019-12-01,2006-01-01,FALSE,0,21,21,75,185,1785,861
13679,Employee Relations Manager,Black,Electronics,Public,FutureWave Corp.,Columbus,Ohio,<2 miles,United States,Non-Travel,...,2024-02-01,2015-01-01,FALSE,0,0,0,0,0,0,0
94202,Software Development Engineer,Hispanic,Financial Services,Cooperative,RuralCooperative Bank,Cleveland,Ohio,>10 miles,United States,Travel Rarely,...,2022-10-01,2020-01-01,TRUE,0,0,0,0,0,0,0
252689,Solutions Architect,Multiple,Technology,Public,SecureTech Solutions,Deltona,Florida,2-5 miles,United States,Non-Travel,...,2022-12-01,2017-01-01,FALSE,0,0,0,0,0,0,0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

In [ ]:
result

In [ ]:
from joblib import dump, load
filename = "HR_Attrition.joblib"
dump(pipeline, filename)

In [ ]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [ ]:
from refractml import *

from refractml.constants import MLModelFlavours

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data_json = eval(payload)
    data = pd.DataFrame([data_json])
    prediction = str(model.predict(data)[0])
    return prediction

In [ ]:
import requests
import datetime
import pandas as pd 
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

In [ ]:
req.json

In [ ]:
## registering the model.
tmp = register_model(pipeline, 
               score, 
               name="HR_ATTRITION_ML_MODEL", 
               description="Analyzing_HR_Attrition_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


In [ ]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

In [ ]:
result = pipeline.predict(X_test)
result_prob = pipeline.predict_proba(X_test)
pred_df = X_test.copy()
result = result
result_prob = result_prob
pred_df["PREDICTION"] = result
pred_df["PROB"] = result_prob[:, 1]

In [ ]:
from sklearn.metrics import classification_report, log_loss, roc_auc_score
 
# Check lengths
print("Length of y_test:", len(y_test))
print("Length of y_pred:", len(y_pred))
 
# Print classification report
print(classification_report(y_test, y_pred))
 
# Calculate accuracy
accuracy = sum(y_test == y_pred) / len(y_test)
print("Accuracy:", accuracy)
 
# Calculate additional metrics
log_loss_value = log_loss(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob[:, 1])  # Assuming class 1 is the positive class
 
print("Log Loss:", log_loss_value)
print("ROC AUC Score:", roc_auc)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns
ns_probs = [0 for _ in range(len(y_test))]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, pred_df["PROB"])
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, pred_df["PROB"])
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Radnomforest')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
lr_precision, lr_recall, _ = precision_recall_curve(y_test, pred_df["PROB"])
plt.plot(lr_recall, lr_precision, marker='.', label='Randomforest')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
cm = confusion_matrix(y_test,y_pred)
 
#Plot the confusion matrix.
sns.heatmap(cm,
            annot=True,
            fmt='g',
            xticklabels=['Not Churn','Churn'],
            yticklabels=['Not CHurn','Churn'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()

In [ ]:
scores_data = pd.DataFrame({"ns_probs":ns_probs,
                            "y_test":y_test,
                            "y_pred":pred_df["PREDICTION"],
                            "act_probs":pred_df["PROB"]})

In [ ]:
scores_data

In [ ]:
scores_data.to_csv("/data/scores.csv", index=False)

In [ ]:
# Define or use the prediction function
def model_prediction_score_func(dataframe):
    # Ensure 'dataframe' has the correct features required by the model
    return pipeline.predict(dataframe)  # Use your trained pipeline/model here
 
# Assuming df is your input DataFrame with the necessary features
Original_df['Model_Output'] = model_prediction_score_func(Original_df)
 
# If you have a DataFrame with test data and you want to merge predictions
# Assuming X_test is the DataFrame for which predictions are made
#X_test_with_predictions = X_test.copy()
#X_test_with_predictions['Model_Output'] = model_prediction_score_func(X_test)
 
# Display the first few rows to verify
#print(X_test_with_predictions.head())

In [ ]:
Original_df

In [ ]:
Original_df.to_csv("/data/Hr_Attrition_Data.csv", index=False)

In [ ]:
con = snowflake.get_connection("HR_ATTRITION")

In [ ]:
import pandas as pd
if 'Model_Output' in Original_df:
    Original_df['Model_Output_Values'] = Original_df['Model_Output'].astype(bool)
print(Original_df.head())


In [ ]:
file = 'HR_Attrition_Data.csv'
Original_df.to_csv(file , index=False)

In [ ]:
file

In [ ]:
snowflake.execute_query(f"PUT file://{file} @HR_ANALYTIC.PUBLIC.HR_ATTRITION_STAGE")